In [280]:
!pip install datasets[vision]
!pip install -Uqq fastai
!pip install huggingface_hub
!pip3 install torch torchvision

In [281]:
from datasets import load_dataset, Image
dataset = load_dataset("jbarat/plant_species", split="train")

Using custom data configuration jbarat--plant_species-efbaed60a3c7fd6e
Reusing dataset parquet (/root/.cache/huggingface/datasets/jbarat___parquet/jbarat--plant_species-efbaed60a3c7fd6e/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


In [282]:
from io import BytesIO
from datasets import Dataset
from PIL import Image

images_dict = {}

for index, example in enumerate(dataset):
    label = str(example["label"])
    image = Image.open(BytesIO(example["image"]["bytes"]))
    if label in images_dict:
        images_dict[label].append(image)
    else:
        images_dict[label] = [image]

dataset = Dataset.from_dict(images_dict)

In [283]:
import torch
from torchvision.transforms import transforms

# Define a function to convert bytes to PILImage
def bytes_to_pil(bytes):
    pil_image = PILImage.open(BytesIO(bytes))
    return pil_image

# Load the dataset
dataset = load_dataset("jbarat/plant_species", split="train")

# Define a custom PyTorch dataset
class PlantSpeciesDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
        self.transform = transforms.Compose([transforms.Resize((224,224)),
                                             transforms.ToTensor(),
                                             transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

    def __getitem__(self, index):
        example = self.dataset[index]
        image_bytes = example['image']['bytes']
        label = example['label']
        pil_image = bytes_to_pil(image_bytes)
        image = self.transform(pil_image)
        return image, label

    def __len__(self):
        return len(self.dataset)

# Create an instance of the custom dataset
custom_dataset = PlantSpeciesDataset(dataset)

# Create a PyTorch DataLoader
dataloader = DataLoader(custom_dataset, batch_size=32, shuffle=True)

# Iterate over the DataLoader
for i, batch in enumerate(dataloader):
    images, labels = batch
    # Do something with the batch of images and labels
    print(f"Batch {i}: Images shape - {images.shape}, Labels shape - {labels.shape}")
